In [ ]:
!pip install gradio -q
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 -f https://download.pytorch.org/whl/torch_stable.html -q
!pip install mamba-ssm==1.2.0.post1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 554.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 14.5 MB/s eta 0:00:00


In [ ]:
import torch
import time
import gradio as gr
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

# ------------------------------------------------------------
# Transformer-based model setup
# ------------------------------------------------------------
transformer_model_path = "hanzla/bert-essay-classifier"
transformer_model = AutoModelForSequenceClassification.from_pretrained(transformer_model_path)
transformer_tokenizer = AutoTokenizer.from_pretrained(transformer_model_path)
transformer_labels = ["Human Written", "AI Written"]

def classify_text_transformer(text):
    start_time = time.time()
    inputs = transformer_tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = transformer_model(**inputs)
    predictions = torch.softmax(outputs.logits, dim=1)
    probabilities = predictions.detach().numpy().flatten() * 100
    computation_time = time.time() - start_time

    # Format results as a Markdown table
    table_str = "| Class | Probability |\n|---|---|\n"
    for i, label in enumerate(transformer_labels):
        table_str += f"| {label} | {probabilities[i]:.2f}% |\n"

    result = (
        f"**Computation Time:** {computation_time:.4f} seconds\n\n"
        "**Prediction Probabilities:**\n\n" + table_str +
        f"\n**Predicted Class:** `{transformer_labels[probabilities.argmax()]}`"
    )
    return result

# ------------------------------------------------------------
# MAMBA-based model setup
# ------------------------------------------------------------
class MambaTextClassifier:
    def __init__(self, model_path, device=None):
        self.device = device if device is not None else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Initialize base model
        self.model = MambaLMHeadModel.from_pretrained("state-spaces/mamba-370m")
        self.model.lm_head = nn.Linear(self.model.config.d_model, 2)

        # Load weights
        state_dict = torch.load(model_path, map_location=self.device)
        if 'state_dict' in state_dict:
            state_dict = state_dict['state_dict']

        self.model.load_state_dict(state_dict)
        self.model.to(self.device)
        self.model.eval()

        self.labels = ['Human-written', 'AI-generated']

    def classify_text(self, text):
        start_time = time.time()

        encoded_input = self.tokenizer(
            text,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        ).to(self.device)

        with torch.no_grad():
            outputs = self.model(encoded_input["input_ids"])
            logits = outputs.logits[:, -1, :]  # last token logits
            probabilities = torch.softmax(logits, dim=1)

        end_time = time.time()
        computation_time = end_time - start_time

        probabilities = probabilities.cpu().numpy().flatten() * 100

        table_str = "| Class | Probability |\n|---|---|\n"
        for i, label in enumerate(self.labels):
            table_str += f"| {label} | {probabilities[i]:.2f}% |\n"

        results = (
            f"**Computation Time:** {computation_time:.4f} seconds\n\n"
            "**Prediction Probabilities:**\n\n" + table_str +
            f"\n**Predicted Class:** `{self.labels[probabilities.argmax()]}`"
        )
        return results

mamba_classifier = MambaTextClassifier("drive/MyDrive/mamba_model/mamba_ai_essay_detection_model_balanced.pth")

def classify_text_mamba(text):
    return mamba_classifier.classify_text(text)

# ------------------------------------------------------------
# Combined function
# ------------------------------------------------------------
def classify_texts(text):
    transformer_result = classify_text_transformer(text)
    mamba_result = classify_text_mamba(text)
    return transformer_result, mamba_result

# ------------------------------------------------------------
# Gradio Interface
# ------------------------------------------------------------
with gr.Blocks(theme=gr.themes.Default()) as demo:
    # Header
    gr.Markdown("<h1 style='text-align: center; margin-bottom: 1em;'>AI vs. Human-Written Text Classifier</h1>", elem_id="title")
    gr.Markdown("<p style='text-align: center; font-size:1.1em;'>Compare the predictions of two different models to determine if text is AI-generated or human-written.</p>")

    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(label="Input Text", lines=5, placeholder="Enter the text you want to classify here...")
            with gr.Row():
                submit_btn = gr.Button("Classify", variant="primary")
                example_btn = gr.Button("Try an Example")

        with gr.Column():
            # Results area
            with gr.Tab("Transformer Model"):
                transformer_output = gr.Markdown(label="Transformer Model Results")
            with gr.Tab("MAMBA Model"):
                mamba_output = gr.Markdown(label="MAMBA Model Results")

    def load_example():
        return """
          Nearly ten years had passed since the Dursleys had woken up to find their nephew on the front step, but Privet Drive had hardly changed at all. The sun rose on the same tidy front gardens and lit up the brass number four on the Dursleys' front door; it crept into their living room, which was almost exactly the same as it had been on the night when Mr. Dursley had seen that fateful news report about the owls. Only the photographs on the mantelpiece really showed how much time had passed. Ten years ago, there had been lots of pictures of what looked like a large pink beach ball wearing different-colored bonnets - but Dudley Dursley was no longer a baby, and now the photographs showed a large blond boy riding his first bicycle, on a carousel at the fair, playing a computer game with his father, being hugged and kissed by his mother. The room held no sign at all that another boy lived in the house, too.

          Yet Harry Potter was still there, asleep at the moment, but not for long. His Aunt Petunia was awake and it was her shrill voice that made the first noise of the day.

          "Up! Get up! Now!"

          Harry woke with a start. His aunt rapped on the door again.

          "Up!" she screeched. Harry heard her walking toward the kitchen and then the sound of the frying pan being put on the stove. He rolled onto his back and tried to remember the dream he had been having. It had been a good one. There had been a flying motorcycle in it. He had a funny feeling he'd had the same dream before.

          His aunt was back outside the door.

          "Are you up yet?" she demanded.

          "Nearly," said Harry.

          "Well, get a move on, I want you to look after the bacon. And don't you dare let it burn, I want everything perfect on Duddy's birthday."

          Harry groaned.

          "What did you say?" his aunt snapped through the door.

          "Nothing, nothing . . ."

          Dudley's birthday - how could he have forgotten? Harry got slowly out of bed and started looking for socks. He found a pair under his bed and, after pulling a spider off one of them, put them on. Harry was used to spiders, because the cupboard under the stairs was full of them, and that was where he slept.

          When he was dressed he went down the hall into the kitchen. The table was almost hidden beneath all Dudley's birthday presents. It looked as though Dudley had gotten the new computer he wanted, not to mention the second television and the racing bike. Exactly why Dudley wanted a racing bike was a mystery to Harry, as Dudley was very fat and hated exercise - unless of course it involved punching somebody. Dudley's favorite punching bag was Harry, but he couldn't often catch him. Harry didn't look it, but he was very fast.

          Perhaps it had something to do with living in a dark cupboard, but Harry had always been small and skinny for his age. He looked even smaller and skinnier than he really was because all he had to wear were old clothes of Dudley's, and Dudley was about four times bigger than he was. Harry had a thin face, knobbly knees, black hair, and bright green eyes. He wore round glasses held together with a lot of Scotch tape because of all the times Dudley had punched him on the nose. The only thing Harry liked about his own appearance was a very thin scar on his forehead that was shaped like a bolt of lightning. He had had it as long as he could remember, and the first question he could ever remember asking his Aunt Petunia was how he had gotten it.

          "In the car crash when your parents died," she had said. "And don't ask questions."

          Don't ask questions - that was the first rule for a quiet life with the Dursleys.

          Uncle Vernon entered the kitchen as Harry was turning over the bacon.

          "Comb your hair!" he barked, by way of a morning greeting.
          """

    example_btn.click(fn=load_example, inputs=[], outputs=[text_input])
    submit_btn.click(
        fn=classify_texts,
        inputs=[text_input],
        outputs=[transformer_output, mamba_output]
    )

    # Footer or additional info in an accordion
    with gr.Accordion("Additional Information"):
        gr.Markdown(
            """
            **Note:**
            - The predictions are probabilistic.
            - The models have been trained on certain datasets and may not be accurate for all types of text.
            - Computation time may vary based on your hardware.
            """
        )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9b413f3aeb7eb90f60.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
